<a href="https://colab.research.google.com/github/Dayaanaly/SA-PU3-Modelo_recurrentes/blob/Esteban/Copia_de_GeneradorTexto002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow==2.0.1 numpy requests tqdm
fhdhe 

SyntaxError: ignored

# preentrenamiento

In [ ]:
import tensorflow as tf
import numpy as np
import os
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from string import punctuation

In [ ]:
import requests
content = requests.get("https://www.gutenberg.org/cache/epub/42324/pg42324.txt").text
open("data\Frankeinstein.txt", "w", encoding="utf-8").write(content)


465593

In [ ]:
sequence_length = 100
BATCH_SIZE = 256
EPOCHS = 30
# dataset file path
FILE_PATH = "data\Frankeinstein.txt"
BASENAME = os.path.basename(FILE_PATH)
# read the data
text = open(FILE_PATH, encoding="utf-8").read()
# remove caps, comment this code if you want uppercase characters as well
#text = text.lower()
# remove punctuation
text = text.translate(str.maketrans("", "", punctuation))
#text = text.ascii_letters + " .,;'"

In [ ]:
# print some stats
n_chars = len(text)
vocab = ''.join(sorted(set(text)))
print("unique_chars:", vocab)
n_unique_chars = len(vocab)
print("Number of characters:", n_chars)
print("Number of unique characters:", n_unique_chars)

unique_chars: 
 0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzæèéêô﻿
Number of characters: 443896
Number of unique characters: 70


In [ ]:
# dictionary that converts characters to integers
char2int = {c: i for i, c in enumerate(vocab)}
# dictionary that converts integers to characters
int2char = {i: c for i, c in enumerate(vocab)}

In [ ]:
# save these dictionaries for later generation
pickle.dump(char2int, open(f"{BASENAME}-char2int.pickle", "wb"))
pickle.dump(int2char, open(f"{BASENAME}-int2char.pickle", "wb"))

In [ ]:
# convert all text into integers
encoded_text = np.array([char2int[c] for c in text])

In [ ]:
# construct tf.data.Dataset object
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [ ]:
# print first 5 characters
for char in char_dataset.take(8):
    print(char.numpy(), int2char[char.numpy()])

69 ﻿
31 T
45 h
42 e
1  
27 P
55 r
52 o


In [ ]:
# build sequences by batching
sequences = char_dataset.batch(2*sequence_length + 1, drop_remainder=True)

# print sequences
for sequence in sequences.take(2):
    print(''.join([int2char[i] for i in sequence.numpy()]))

﻿The Project Gutenberg EBook of Frankenstein by Mary W Shelley

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever  You may copy it give it away or
reuse
 it under the terms of the Project Gutenberg License included
with this eBook or online at wwwgutenbergorg


Title Frankenstein
       or The Modern Prometheus

Author Mary W Shelley

Release Date Marc


In [ ]:
def split_sample(sample):
    # example :
    # sequence_length is 10
    # sample is "python is a great pro" (21 length)
    # ds will equal to ('python is ', 'a') encoded as integers
    ds = tf.data.Dataset.from_tensors((sample[:sequence_length], sample[sequence_length]))
    for i in range(1, (len(sample)-1) // 2):
        # first (input_, target) will be ('ython is a', ' ')
        # second (input_, target) will be ('thon is a ', 'g')
        # third (input_, target) will be ('hon is a g', 'r')
        # and so on
        input_ = sample[i: i+sequence_length]
        target = sample[i+sequence_length]
        # extend the dataset with these samples by concatenate() method
        other_ds = tf.data.Dataset.from_tensors((input_, target))
        ds = ds.concatenate(other_ds)
    return ds

# prepare inputs and targets
dataset = sequences.flat_map(split_sample)

In [ ]:
def one_hot_samples(input_, target):
    # onehot encode the inputs and the targets
    # Example:
    # if character 'd' is encoded as 3 and n_unique_chars = 5
    # result should be the vector: [0, 0, 0, 1, 0], since 'd' is the 4th character
    return tf.one_hot(input_, n_unique_chars), tf.one_hot(target, n_unique_chars)

dataset = dataset.map(one_hot_samples)

In [ ]:
# print first 2 samples
for element in dataset.take(2):
    print("Input:", ''.join([int2char[np.argmax(char_vector)] for char_vector in element[0].numpy()]))
    print("Target:", int2char[np.argmax(element[1].numpy())])
    print("Input shape:", element[0].shape)
    print("Target shape:", element[1].shape)
    print("="*50, "\n")

Input: ﻿The Project Gutenberg EBook of Frankenstein by Mary W Shelley

This eBook is for the use of anyone 
Target: a
Input shape: (100, 70)
Target shape: (70,)

Input: The Project Gutenberg EBook of Frankenstein by Mary W Shelley

This eBook is for the use of anyone a
Target: n
Input shape: (100, 70)
Target shape: (70,)



In [ ]:
# repeat, shuffle and batch the dataset
ds = dataset.repeat().shuffle(1024).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
pip install -U numpy==1.18.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
model = Sequential([
    LSTM(256, input_shape=(sequence_length, n_unique_chars), return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dense(n_unique_chars, activation="softmax"),])

In [ ]:
# define the model path
model_weights_path = f"results/{BASENAME}-{sequence_length}.h5"
model.summary()
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 100, 256)          334848    
                                                                 
 dropout_1 (Dropout)         (None, 100, 256)          0         
                                                                 
 lstm_3 (LSTM)               (None, 256)               525312    
                                                                 
 dense_1 (Dense)             (None, 70)                17990     
                                                                 
Total params: 878,150
Trainable params: 878,150
Non-trainable params: 0
_________________________________________________________________


# entrenamiento

In [ ]:
# make results folder if does not exist yet
if not os.path.isdir("results"):
    os.mkdir("results")
# train the model
model.fit(ds, steps_per_epoch=(len(encoded_text) - sequence_length) // BATCH_SIZE, epochs=EPOCHS)
# save the model
model.save(model_weights_path)

Epoch 1/30
1733/1733 [==============================] - 132s 72ms/step - loss: 2.6528 - accuracy: 0.2509
Epoch 2/30
1733/1733 [==============================] - 124s 71ms/step - loss: 2.2179 - accuracy: 0.3525
Epoch 3/30
1733/1733 [==============================] - 124s 71ms/step - loss: 1.8687 - accuracy: 0.4404
Epoch 4/30
1733/1733 [==============================] - 124s 72ms/step - loss: 1.6745 - accuracy: 0.4936
Epoch 5/30
1733/1733 [==============================] - 124s 71ms/step - loss: 1.5501 - accuracy: 0.5318
Epoch 6/30
1733/1733 [==============================] - 124s 72ms/step - loss: 1.4651 - accuracy: 0.5568
Epoch 7/30
1733/1733 [==============================] - 124s 72ms/step - loss: 1.4032 - accuracy: 0.5747
Epoch 8/30
1733/1733 [==============================] - 125s 72ms/step - loss: 1.3560 - accuracy: 0.5873
Epoch 9/30
1733/1733 [==============================] - 125s 72ms/step - loss: 1.3169 - accuracy: 0.5976
Epoch 10/30
1733/1733 [==============================] 

In [ ]:
import numpy as np
import pickle
import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Activation
import os

sequence_length = 100
# dataset file path
FILE_PATH = "/content/data\Frankeinstein.txt"
# FILE_PATH = "data/python_code.py"
BASENAME = os.path.basename(FILE_PATH)

In [ ]:
seed = "disconfort"

In [ ]:
# load vocab dictionaries
char2int = pickle.load(open(f"{BASENAME}-char2int.pickle", "rb"))
int2char = pickle.load(open(f"{BASENAME}-int2char.pickle", "rb"))
vocab_size = len(char2int)

In [ ]:
# building the model
model = Sequential([
    LSTM(256, input_shape=(sequence_length, vocab_size), return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dense(vocab_size, activation="softmax"),
])

In [ ]:
# load the optimal weights
model.load_weights(f"results/{BASENAME}-{sequence_length}.h5")

In [ ]:
s = seed
n_chars = 5000
# generate 400 characters
generated = ""
for i in tqdm.tqdm(range(n_chars), "Generating text"):
    # make the input sequence
    X = np.zeros((1, sequence_length, vocab_size))
    for t, char in enumerate(seed):
        X[0, (sequence_length - len(seed)) + t, char2int[char]] = 1
    # predict the next character
    predicted = model.predict(X, verbose=0)[0]
    # converting the vector to an integer
    next_index = np.argmax(predicted)
    # converting the integer to a character
    next_char = int2char[next_index]
    # add the character to results
    generated += next_char
    # shift seed and the predicted character
    seed = seed[1:] + next_char

print("Seed:", s)
print("Generated text:")
print(generated)

Generating text: 100%|██████████| 5000/5000 [03:49<00:00, 21.80it/s]

Seed: disconfort
Generated text:


It is concerning
to me I should
having last
elevating a
very diver
by the more
smiled her
could have
been that a
shap of her
countenance
and he felt
because the
search for
ever than his
friends which
provided his
encounted and
the anguared
by the more
smiled her
could have
been that a
shap of her
countenance
and he felt
because the
search for
ever than his
friends which
provided his
encounted and
the anguared
by the more
smiled her
could have
been that a
shap of her
countenance
and he felt
because the
search for
ever than his
friends which
provided his
encounted and
the anguared
by the more
smiled her
could have
been that a
shap of her
countenance
and he felt
because the
search for
ever than his
friends which
provided his
encounted and
the anguared
by the more
smiled her
could have
been that a
shap of her
countenance
and he felt
because the
search for
ever than his
friends which
provided his
encounted and
the anguared
by the more
smiled her
could have